Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по правилам:

оценка 2 и ниже — низкий рейтинг;
оценка 4 и ниже — средний рейтинг;
оценка 4.5 и 5 — высокий рейтинг.
Результат классификации запишите в столбец class.

In [1]:
import pandas as pd

In [3]:
df_movies = pd.read_csv('movies.csv')
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
df_ratings = pd.read_csv('ratings.csv')
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [5]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [6]:
def func_rating(row):
    if row['rating'] <= 2.0:
        return 'Низкий рейтинг'
    elif row['rating'] > 2.0 and row['rating'] <= 4.0:
        return 'Средний рейтинг'
    else:
        return 'Высокий рейтинг'


In [39]:
df_new = df_ratings.groupby('movieId').mean('rating').sort_values(by='movieId', ascending=False)[['rating']]
# формирует новый датафрейм, оставляя только id фильма и класс, другие столбцы нам не нужны. 
df_new['class'] = df_new.apply(func_rating, axis=1)
# добавляем колонку класс
df_new.head(10)

,rating,class
movieId,,
163949,5.0,Высокий рейтинг
162672,3.0,Средний рейтинг
162542,5.0,Высокий рейтинг
162376,4.5,Высокий рейтинг
161944,5.0,Высокий рейтинг
161918,1.5,Низкий рейтинг
161830,1.0,Низкий рейтинг
161594,3.0,Средний рейтинг
161155,0.5,Низкий рейтинг


In [ ]:
df_new.head(15)

In [16]:
joined = df_movies.merge(df_new, on='movieId', how='left')
# Чтобы увидеть полную информацию по каждому фильму с указанием класса, обогатим таблицу фильмов, расчитанными ранее классами
joined.head(10)

,movieId,title,genres,rating,class
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.872470,Средний рейтинг
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.401869,Средний рейтинг
2,3,Grumpier Old Men (1995),Comedy|Romance,3.161017,Средний рейтинг
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.384615,Средний рейтинг
4,5,Father of the Bride Part II (1995),Comedy,3.267857,Средний рейтинг
5,6,Heat (1995),Action|Crime|Thriller,3.884615,Средний рейтинг
6,7,Sabrina (1995),Comedy|Romance,3.283019,Средний рейтинг
7,8,Tom and Huck (1995),Adventure|Children,3.800000,Средний рейтинг
8,9,Sudden Death (1995),Action,3.150000,Средний рейтинг
9,10,GoldenEye (1995),Action|Adventure|Thriller,3.450820,Средний рейтинг


Задание 2
Используйте файл keywords.csv.

Нужно написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определённому региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

In [2]:
geo_data = {
'Центр': ['вк','москва', 'тула', 'ярославль'],       # чтобы результат работы функции был виден в первых строк, занесен 'вк' 
'Северо-Запад': ['порно','петербург', 'псков', 'мурманск'],     # и 'порно'
'Дальний Восток': ['вконтакте', 'владивосток', 'сахалин', 'хабаровск']
}

In [3]:
df_keywords = pd.read_csv('keywords.csv')
df_keywords.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [4]:
df = pd.read_csv('keywords.csv')
df.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [5]:
def geo1(row):                        # 1 вариант ручного внесения справочника. Если справочник большой - не подойдет.
    for word in row['keyword'].split():
        if word in ['вк','москва', 'тула', 'ярославль']:
            return 'Центр'
        elif word in ['порно','петербург', 'псков', 'мурманск']:
            return 'Северо-Запад'
        elif word in ['вконтакте','владивосток', 'сахалин', 'хабаровск']:
            return 'Дальний Восток'
        else:
            return 'undefined'

In [6]:
def geo2(row):                        # 2 вариант(основной) разбора любого размера справочника.
    for word in row['keyword'].split():
        for key,values in geo_data.items():
            if word in values:
                return key
        else:
            return 'undefined'

In [46]:
def geo3(row):                        # 3 вариант
    for word in row['keyword'].split():
        for key,values in geo_data.items():
            if word in values:
                return key
    return 'undefined'

In [8]:
df_keywords['region'] = df_keywords.apply(geo1, axis = 1)   # 1 вариант - решение ()
df_keywords.head(20)

,keyword,shows,region
0,вк,64292779,Центр
1,одноклассники,63810309,undefined
2,порно,41747114,Северо-Запад
3,ютуб,39995567,undefined
4,вконтакте,21014195,Дальний Восток
5,одноклассники моя страница,19976138,undefined
6,майл,15144673,undefined
7,авито,14786593,undefined
8,переводчик,14453469,undefined
9,яндекс,13344734,undefined


In [42]:
df_keywords['region'] = df_keywords.apply(geo2, axis = 1)   # 2 вариант (основной) - решение
df_keywords.head(20)

,keyword,shows,region
0,вк,64292779,Центр
1,одноклассники,63810309,undefined
2,порно,41747114,Северо-Запад
3,ютуб,39995567,undefined
4,вконтакте,21014195,Дальний Восток
5,одноклассники моя страница,19976138,undefined
6,майл,15144673,undefined
7,авито,14786593,undefined
8,переводчик,14453469,undefined
9,яндекс,13344734,undefined


In [47]:
df_keywords['region'] = df_keywords.apply(geo3, axis = 1)   # 3 
df_keywords.head(20)

,keyword,shows,region
0,вк,64292779,Центр
1,одноклассники,63810309,undefined
2,порно,41747114,Северо-Запад
3,ютуб,39995567,undefined
4,вконтакте,21014195,Дальний Восток
5,одноклассники моя страница,19976138,undefined
6,майл,15144673,undefined
7,авито,14786593,undefined
8,переводчик,14453469,undefined
9,яндекс,13344734,undefined


In [33]:
kd=pd.DataFrame({'keyword': ['вк москва']})
kd

,keyword
0,вк москва


In [44]:
kd['reg']=kd.apply(geo1, axis = 1)
kd

,keyword,reg
0,вк москва,Центр
